In [ ]:
import os
import slideio
import numpy as np
from pathlib import Path
import cv2
import csv
import time

from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, Input, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from PIL import Image
from itertools import product

# Path to the main folder containing sub-folders of tiles
main_folder = 'C:/Users/kapongo.lumamba/TB_Demo/WSI-SVS-Tiles_v2'  

### Function to create the prediction model ###########################################
def create_model_from_weights(model_file):
    # The model is based on Inceptionv3 architecture, with changed classification layer
    RANDOM_SEED = 27
    model = keras.applications.InceptionV3(include_top=False, input_tensor=Input(shape=(224, 224, 3)))

    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1, seed=RANDOM_SEED)(x)
    predictions = Dense(2, activation='softmax')(x)

    model = Model(inputs=model.input, outputs=predictions)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.load_weights(model_file)

    return model

### Function to normalise images ######################################################
def read_normalized_image(file_name):
    # Check if file exists
    if not os.path.isfile(file_name):
        print(f"File '{file_name}' does not exist.")
        return None
    
    img = cv2.imread(file_name)
    
    # Check if image was successfully loaded
    if img is None:
        print(f"Failed to load image: {file_name}")
        return None
    
    # Changing the order of colour to keep the same ordering used while training
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Rescaling to network input resolution
    img = cv2.resize(img, (224, 224))
    # Standardization of pixel values
    mean = np.mean(img, axis=(0,1), keepdims=True)
    std = np.sqrt(((img - mean) ** 2).mean(axis=(0,1)))
    img = (img - mean) / (std + 0.000001)

    return img

### Function to create CSV file for each sub-folder ########################################
def create_csv_for_subfolder(subfolder_path):
    # Extract sub-folder name from its path
    subfolder_name = os.path.basename(subfolder_path)
    # Define CSV file path for the sub-folder
    csv_file_path = os.path.join(subfolder_path, f"{subfolder_name}.csv")
    
    # Write to CSV file
    with open(csv_file_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['File', 'Type', 'Score'])  # Header row   
        
    print(f"CSV file created for sub-folder '{subfolder_name}' at: {csv_file_path}")

### Function to read and analyse the tiles ###############################################
def process_files_in_subfolder(subfolder_path, model):
    # Initialize list to store rows for CSV
    rows = [] 
    # Define CSV file path for the sub-folder
    subfolder_name = os.path.basename(subfolder_path)
    csv_file_path = os.path.join(subfolder_path, f"{subfolder_name}.csv")

    # Iterate through files in the sub-folder (assuming only .png files)
    for filename in os.listdir(subfolder_path):
        if filename.endswith('.png'):
            file_path = os.path.join(subfolder_path, filename)
            
            # Process the .png file here (e.g., read, analyze, etc.)
            img = read_normalized_image(str(file_path))
            org = cv2.imread(str(file_path))
            predicted = model.predict(np.expand_dims(img, axis=0))
            predicted_class = np.argmax(predicted)
            predicted_score = predicted[0][predicted_class]

            # Construct the line to be written in CSV
            line = "{file} is {type} with score {score}".format(file=filename,
                type="POSITIVE" if predicted_class == 1 else "NEGATIVE",
                score=predicted_score
            )
        
            # Append to the rows list
            rows.append([filename, "POSITIVE" if predicted_class == 1 else "NEGATIVE", predicted_score])
        else:
            print("Not a .png image - process_files_in_subfolder.")
            
    # Append data to CSV file
    with open(csv_file_path, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerows(rows)
           

### Main Function ####################################################################################
def main():
    #function call to create model
    model_file = "fold1_bestepoch4_bestacc0.9848_True.h5"
    model = create_model_from_weights(model_file=model_file)
    
    # Iterate through sub-folders in the main folder
    for subdir in os.listdir(main_folder):
        subfolder_path = os.path.join(main_folder, subdir)
        if os.path.isdir(subfolder_path):
            print(f"Processing sub-folder: {subfolder_path}")
            
            
            create_csv_for_subfolder(subfolder_path) ### Function call to create a csv file
            process_files_in_subfolder(subfolder_path, model) ### Function call to read and process the png files
        else:
            print("Incorrect path...")

if __name__ == "__main__":
    main()

Processing sub-folder: C:/Users/ga-steynlab-03/Demo_TB/wsi_tiles_analysis\SL06-D36-2
CSV file created for sub-folder 'SL06-D36-2' at: C:/Users/ga-steynlab-03/Demo_TB/wsi_tiles_analysis\SL06-D36-2\SL06-D36-2.csv
Not a .png image - process_files_in_subfolder.
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━